In [0]:
!pip install pydrive

In [0]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim


##### START OF ADDITION OF MY CODE

import numpy as np
import os.path
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
from googleapiclient.http import MediaIoBaseDownload

from scipy.io import loadmat
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.io as sio
import sys

## PyTorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import Tensor
import torch.optim as optim
import torchvision

import math #for calculus


In [0]:

## END OF IMPORTS
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#file_id = '1FrXb6rTyqpE5SmNtP8HhygDmFf9Lw896'
file_id = '1al08tF3j-Z2-fowjPPNz1SrFgLVqywXG' ##1.mat
#https://drive.google.com/open?id=1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx
file_id = '1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx'
#https://drive.google.com/open?id=1al08tF3j-Z2-fowjPPNz1SrFgLVqywXG
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('./train3.mat')
#mat = loadmat('train3.mat')
mat = sio.loadmat('train3.mat', squeeze_me=True, struct_as_record=False)
data=[]

## in class      
device='cuda:0'
learning_rate=0.01
weight_decay=0.000001 
momentum=0.9
batch_size=200

o=mat['o']

data=mat['o'].data
#print(data[200][21])
labels=mat['o'].marker

newlabels = np.zeros([data.shape[0],4])

#print(newlabels.shape)
#print(labels.shape)

#print(np.amax(labels))

found = np.zeros([1,])
#print(found.shape)

#clean the shit out of it
while np.amax(labels) > 3:
    ind = np.argmax(labels)
    found = np.append(found,np.array([ind]), axis=0)
    labels[ind] = 0
    data[ind,:] = np.zeros([22,])
    
    


#print(np.amax(labels))
#print(found.shape)
#print(found)

inside = 0
for y in labels:
  if inside % 10000 == 0:
    #print(inside)
    newlabels[inside,y] = 1
    inside += 1
  
#print(newlabels.shape)



In [0]:
## video maker and data extractor

pos=np.zeros((22,3), dtype=int)

pos[0]=-2,10,0
pos[1]=2,10,0
pos[3]=-4,7,3
pos[4]=4,7,3
pos[5]=-5,0,4
pos[6]=5,0,4
pos[7]=-4,-7,3
pos[8]=4,-7,3
pos[9]=-2,-10,0
pos[10]=2,-10,0
pos[11]=-7,0,-5
pos[12]=7,0,5
pos[13]=-6,6,0
pos[14]=6,6,0
pos[15]=-7,0,0
pos[16]=7,0,0
pos[17]=-6,-6,0
pos[18]=6,-6,0
pos[19]=0,7,4
pos[20]=0,0,6
pos[21]=0,-7,4

def make_3d_point(x,y,z,r,theta,phi):
  return [
      x + int(r * math.cos(theta)*math.sin(phi)),
      y + int(r * math.sin(theta)*math.sin(phi)),
      z + int(r * math.cos(phi))
         ]

def make_3d_image(image3d_data,shift):
  
 # print("started a 3d image")
  
  #creating the matrix
  matrix = np.zeros([12,16,22]) # z,x,y
  
  #looping through data
  for point in image3d_data:
    
    # creating the indice by shifting the coordinate system from the center to left corner
    newpoint = [int(point[0]) + 7,int(point[1]) + 10,int(point[2]) + 5]
    
    if newpoint[0] >= matrix.shape[1] or newpoint[1] >= matrix.shape[2] or newpoint[2] >= matrix.shape[0] or newpoint[0] < 0 or newpoint[1] < 0 or newpoint[2] < 0:
      print("did not add point")
      print(newpoint)
      print(matrix.shape)
      continue

      
    #the maximum value
    cap = 400 + shift #based on the data (a peak - maximum value is equal or higher than 400)
    
    #for scaling the values between 0 and 255
    scale = 255/cap
    
    #assigning the value
    if int(point[3]) >= 0 and int(point[3]) <= cap:
      if matrix[newpoint[2],newpoint[0],newpoint[1]] == 0 or matrix[newpoint[2],newpoint[0],newpoint[1]] < int(point[3]*scale):
        matrix[newpoint[2],newpoint[0],newpoint[1]] = int(point[3]*scale)

    elif int(point[3]) > cap:
      if matrix[newpoint[2],newpoint[0],newpoint[1]] == 0 or matrix[newpoint[2],newpoint[0],newpoint[1]] < int(cap*scale):
        matrix[newpoint[2],newpoint[0],newpoint[1]] = int(cap*scale)
    else:
      if matrix[newpoint[2],newpoint[0],newpoint[1]] < 0:
        matrix[newpoint[2],newpoint[0],newpoint[1]] = 0
    
    #creating variables for creating a virtual sphere around poi
    rho = [2,3] # distance (in pixels) that we are interpolating from poi
    #theta  angle in radians    [0,2pi]
    #phi    angle in radians    [0, pi]
    
    #creating new interpolated points
    for r in rho:
      for theta in range(0,360,45):
        for phi in range(0,180,45):
          p = make_3d_point(newpoint[0],newpoint[1],newpoint[2],r,math.radians(theta),math.radians(phi))
          
          #checking if its in bounds
          if p[0] < matrix.shape[1] and p[1] < matrix.shape[2] and p[2] < matrix.shape[0] and p[0] >= 0 and p[1] >= 0 and p[2] >= 0:

            
            #assigning the value to the matrix, scaled down by distance
            if int(point[3]) >= 0 and int(point[3]) <= cap:
              if matrix[p[2],p[0],p[1]] == 0:
                matrix[p[2],p[0],p[1]] = int(point[3]/r*scale)
            elif int(point[3]) > cap:
              if matrix[p[2],p[0],p[1]] == 0:
                matrix[p[2],p[0],p[1]] = int(cap/r*scale)
            else:
              if matrix[p[2],p[0],p[1]] == 0:
                matrix[p[2],p[0],p[1]] = 0
            
  return matrix


start = 0
end   = 3335
#start = 0
#end = 667000
step = 1

#shift the data because it has negative values
shift_data = 300
\
newdata = np.add(data,shift_data).reshape([22,data.shape[0]])

video_stream = np.zeros([end-start,1,12,16,22])
video_stream[0,0] = make_3d_image(np.hstack((pos,newdata[:,start].reshape([22,1]))),shift_data)

print(np.hstack((pos,newdata[:,start].reshape([22,1]))))


"""for t in range(start+step,end,step):
 
  #create x,y,z,value for timestep t
  data3d = np.hstack((pos,newdata[:,t].reshape([22,1])))
  
  #make image - with interpolated data for timestep t
  result = make_3d_image(data3d,shift_data)
  
  #add image to video stream
  video_stream[t-start,0] = result
  
  
maxValue = np.amax(video_stream)
minValue = np.amin(video_stream)"""

#print(maxValue)
#print(minValue)
#print(video_stream[0,0,3])

[[ -2.    10.     0.   300.  ]
 [  2.    10.     0.   302.51]
 [  0.     0.     0.   294.96]
 [ -4.     7.     3.   290.72]
 [  4.     7.     3.   323.97]
 [ -5.     0.     4.   292.55]
 [  5.     0.     4.   301.08]
 [ -4.    -7.     3.   300.43]
 [  4.    -7.     3.   300.  ]
 [ -2.   -10.     0.   306.18]
 [  2.   -10.     0.   306.36]
 [ -7.     0.    -5.   270.89]
 [  7.     0.     5.   303.68]
 [ -6.     6.     0.   290.32]
 [  6.     6.     0.   315.5 ]
 [ -7.     0.     0.   300.  ]
 [  7.     0.     0.   292.41]
 [ -6.    -6.     0.   299.85]
 [  6.    -6.     0.   291.02]
 [  0.     7.     4.   295.93]
 [  0.     0.     6.   283.45]
 [  0.    -7.     4.   300.77]]


'for t in range(start+step,end,step):\n \n  #create x,y,z,value for timestep t\n  data3d = np.hstack((pos,newdata[:,t].reshape([22,1])))\n  \n  #make image - with interpolated data for timestep t\n  result = make_3d_image(data3d,shift_data)\n  \n  #add image to video stream\n  video_stream[t-start,0] = result\n  \n  \nmaxValue = np.amax(video_stream)\nminValue = np.amin(video_stream)'

In [0]:
class eeg_CNN(nn.Module):
    def __init__(self):
        super(eeg_CNN, self).__init__()
        self.T = 120
        
        self.conv1 = nn.Conv3d(1,5,2, stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm1 = nn.BatchNorm3d(5)
        self.conv2 = nn.Conv3d(5,10,(2,3,2), stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm2 = nn.BatchNorm3d(10)
        self.conv3 = nn.Conv3d(10,5,3, stride=1,dilation=1)
        self.batchnorm3 = nn.BatchNorm3d(5)
        self.conv4 = nn.Conv3d(5,5,(3,3,2), stride=1, dilation =1)
        self.batchnorm4 = nn.BatchNorm3d(5)
        self.conv5 = nn.Conv3d(5,5,(1,2,1), stride=1, dilation =1)
        self.batchnorm5 = nn.BatchNorm3d(5)
        self.pooling1 = nn.MaxPool3d((2,2,2), stride=1, dilation=1)
        
        #layer after max pool
        self.conv6 = nn.Conv3d(5,5,3, stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm6 = nn.BatchNorm3d(5)
        self.conv7 = nn.Conv3d(5,5,(3,3,2), stride=1,dilation=1)
        self.batchnorm7 = nn.BatchNorm3d(5)
        self.conv8 = nn.Conv3d(5,5,(1,2,1), stride=1,dilation=1)
        self.batchnorm8 = nn.BatchNorm3d(5)
        self.pooling2 = nn.MaxPool3d((3,3,2), stride=1, dilation=1)
        
        #Layer after next max pool 
        self.conv9 = nn.Conv3d(5,5,3, stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm9 = nn.BatchNorm3d(5)
        self.conv10 = nn.Conv3d(5,5,(3,3,2), stride=1,dilation=1)
        self.batchnorm10 = nn.BatchNorm3d(5)
        self.conv11 = nn.Conv3d(5,1,(1,2,1), stride=1,dilation=1)
        self.batchnorm11 = nn.BatchNorm3d(5)
        self.pooling3 = nn.MaxPool3d((3,3,2), stride=1, dilation=1)

        
    def forward(self, x):
      
        # first set of CNNs and then a max pool
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.relu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.relu(self.conv3(x)) 
        x = self.batchnorm3(x)
        x = F.relu(self.conv4(x))
        x = self.batchnorm4(x)
        x = F.relu(self.conv5(x))
        x = self.batchnorm5(x)
        x = self.pooling1(x)
        
        # second set
        
        x = F.relu(self.conv6(x))
        x = self.batchnorm6(x)
        x = F.relu(self.conv7(x))
        x = self.batchnorm7(x)
        x = F.relu(self.conv8(x))
        x = self.batchnorm9(x)
        x = self.pooling2(x)
        
        # set 3  
        
        x = F.relu(self.conv9(x))
        x = self.batchnorm10(x)
        x = F.relu(self.conv10(x))
        x = self.batchnorm11(x)
        x = F.relu(self.conv11(x))
        x = self.pooling3(x)
        batch_size, timesteps, C, H, W = x.size()
        c_in = x.view(batch_size * timesteps, C, H, W)
        c_out = c_in[:,0,:,:]
        return c_out
      
      
      
class eeg_LSTM(nn.Module):
    def __init__(self):
        super(eeg_LSTM, self).__init__()
        
        #self.fc1 = nn.Linear(16 * 1 * 1, 10)
        ## here is the start of the LSTM 
        self.rnn = nn.LSTM(16, 4, 2)
        #self.fc1 = nn.Linear(4,4)

    def forward(self, x):
        C, H, W = x.size()
        #print("shape in forward",x.shape)
        c_in = x.view(C, H, W)
        print("shape in forward",x.shape)
        ##c_in = c_in[:,0,:,:]
        r_out, _ = self.rnn(c_in)
   
        output = F.softmax(r_out) 
        return output


In [0]:


class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        self.conv1 = nn.Conv3d(1,5,2, stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm1 = nn.BatchNorm3d(5)
        self.conv2 = nn.Conv3d(5,10,(2,3,2), stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm2 = nn.BatchNorm3d(10)
        self.conv3 = nn.Conv3d(10,5,3, stride=1,dilation=1)
        self.batchnorm3 = nn.BatchNorm3d(5)
        self.conv4 = nn.Conv3d(5,5,(3,3,2), stride=1, dilation =1)
        self.batchnorm4 = nn.BatchNorm3d(5)
        self.conv5 = nn.Conv3d(5,5,(1,2,1), stride=1, dilation =1)
        self.batchnorm5 = nn.BatchNorm3d(5)
        self.pooling1 = nn.MaxPool3d((2,2,2), stride=1, dilation=1)
        
        #layer after max pool
        self.conv6 = nn.Conv3d(5,5,3, stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm6 = nn.BatchNorm3d(5)
        self.conv7 = nn.Conv3d(5,5,(3,3,2), stride=1,dilation=1)
        self.batchnorm7 = nn.BatchNorm3d(5)
        self.conv8 = nn.Conv3d(5,5,(1,2,1), stride=1,dilation=1)
        self.batchnorm8 = nn.BatchNorm3d(5)
        self.pooling2 = nn.MaxPool3d((3,3,2), stride=1, dilation=1)
        
        #Layer after next max pool 
        self.conv9 = nn.Conv3d(5,5,3, stride=1,padding=(1,1,1),dilation=1)
        self.batchnorm9 = nn.BatchNorm3d(5)
        self.conv10 = nn.Conv3d(5,5,(3,3,2), stride=1,dilation=1)
        self.batchnorm10 = nn.BatchNorm3d(5)
        self.conv11 = nn.Conv3d(5,1,(1,2,1), stride=1,dilation=1)
        self.batchnorm11 = nn.BatchNorm3d(5)
        self.pooling3 = nn.MaxPool3d((3,3,2), stride=1, dilation=1)
        
        #self.fc1 = nn.Linear(16 * 1 * 1, 10)
        ## here is the start of the LSTM 
        self.rnn = nn.LSTM(16, 4, 2)
        #self.fc1 = nn.Linear(4,4)
        
        
    def forward(self, x):
      
        # first set of CNNs and then a max pool
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.relu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.relu(self.conv3(x)) 
        x = self.batchnorm3(x)
        x = F.relu(self.conv4(x))
        x = self.batchnorm4(x)
        x = F.relu(self.conv5(x))
        x = self.batchnorm5(x)
        x = self.pooling1(x)
        
        # second set
        
        x = F.relu(self.conv6(x))
        x = self.batchnorm6(x)
        x = F.relu(self.conv7(x))
        x = self.batchnorm7(x)
        x = F.relu(self.conv8(x))
        x = self.batchnorm9(x)
        x = self.pooling2(x)
        
        # set 3  
        
        x = F.relu(self.conv9(x))
        x = self.batchnorm10(x)
        x = F.relu(self.conv10(x))
        x = self.batchnorm11(x)
        x = F.relu(self.conv11(x))
        x = self.pooling3(x)
        
 

        #Start of LSTM
        batch_size, timesteps, C, H, W = x.size()
        c_in = x.view(batch_size * timesteps, C, H, W)
        c_in = c_in[:,0,:,:]
        #print("stsrting")
        #print(c_in.shape)
        
        r_out, _ = self.rnn(c_in)
        #print(r_out.shape)
        #print("endsing")
        
        #t =  self.fc1(r_out)
        #t = F.log_softmax(r_out, dim=0)
        output = F.softmax(r_out)#, dim=0)
        return output
      
## END OF ADDITION OF MY CODE
      
device='cuda:0'            
net = EEGNet().to(device)
preNet = eeg_CNN().to(device)
lstm = eeg_LSTM().to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.08, momentum=0.9)
loss = nn.CrossEntropyLoss()

#X_train = video_stream.astype('float32') # our generated image
#y_train = torch.from_numpy(labels.reshape(667000,1)[start:end])#()2668000,1

y_train = torch.from_numpy(labels.reshape(667000,1))#()2668000,1


batch_size = 115

video_batch=29

begin=0
finish=3335

step=0
t=0
print(begin, finish, step)
#print(y_train.max())
#for epoch in range(0,X_train.shape[0],batch_size):  # loop over the dataset multiple times ##
for epoch in range(0,200):
  print("\nEpoch ", epoch)
  running_loss = 0.0
  
  for t in range(begin,finish,1):
    #create x,y,z,value for timestep t
    data3d = np.hstack((pos,newdata[:,t].reshape([22,1])))
    #make image - with interpolated data for timestep t
    result = make_3d_image(data3d,shift_data)
    #add image to video stream
    video_stream[t-begin,0] = result
  
  
  print("vid stream shape is ", video_stream.shape)
  x_train = video_stream.astype('float32')# our generated image
  y_train = torch.from_numpy(labels[begin:finish].reshape(finish-begin,1))#()2668000,1
  begin = finish
  step = 3335
  t = t+step
  finish = finish+step
  print("next index begin is ", begin, "finish is ", finish)
  
  print("Lets preprocess our batch of CNN images ")
  vid_cnn = torch.zeros(video_batch,1,16)
  vid_cnn.double()

  
  for i in range((len(x_train)//batch_size)-1):
        s = i*1
        e = i*1+1
        preProcess = Variable(torch.tensor(x_train[s:e])).to(device)
        out = preNet(preProcess)
        #print("shape of out is ", out.shape)
        vid_cnn[i] = out
        
  print("vi_cnn shape is ",vid_cnn.shape)
  
  print("Train on LSTM of processed CNN vids")
  for i in range((len(x_train)//batch_size)-1):   
        optimizer.zero_grad()
        s = i*batch_size
        e = i*batch_size+batch_size
        train = Variable(vid_cnn).to(device)
        out = lstm(train)
        last_output = out[-1]

        target = Variable(torch.LongTensor([y_train[0:e-s].max()])).to(device)


        err = loss(last_output, target)
        err.backward()#retain_graph=True
        optimizer.step()
        if i % 6 == 0:
          print("target ", target)
          print("Loss is ",err , i, epoch)
          print("Last output ",last_output[-1])
  
  
  
"""
  for i in range((len(X_train)//batch_size)-1):   
        optimizer.zero_grad()
        s = i*batch_size
        e = i*batch_size+batch_size
        train = Variable(torch.tensor(X_train[s:e])).to(device)

        out = net(train)
        last_output = out[-1]

        target = Variable(torch.LongTensor([y_train[0:e-s].max()])).to(device)

        #print(target)
        #print(last_output[-1])
        err = loss(last_output, target)
        err.backward()#retain_graph=True
        optimizer.step()
        if i % 6 == 0:
          print("target ", target)
          print("Loss is ",err , i, epoch)
          print("Last output ",last_output[-1])"""

0 3335 0

Epoch  0
vid stream shape is  (3335, 1, 12, 16, 22)
next index begin is  3335 finish is  6670
Lets preprocess our batch of CNN images 
vi_cnn shape is  torch.Size([29, 1, 16])
Train on LSTM of processed CNN vids
shape in forward torch.Size([29, 1, 16])
target  tensor([0], device='cuda:0')
Loss is  tensor(1.3865, device='cuda:0', grad_fn=<NllLossBackward>) 0 0
Last output  tensor([0.0347, 0.0351, 0.0352, 0.0345], device='cuda:0',
       grad_fn=<SelectBackward>)
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
target  tensor([0], device='cuda:0')
Loss is  tensor(1.3865, device='cuda:0', grad_fn=<NllLossBackward>) 6 0
Last output  tensor([0.0347, 0.0351, 0.0352, 0.0345], device='cuda:0',
       grad_fn=<SelectBackward>)
shape in forward torch.Size([29, 1, 16])
shape in forward torch.S

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
target  tensor([0], device='cuda:0')
Loss is  tensor(1.3865, device='cuda:0', grad_fn=<NllLossBackward>) 12 0
Last output  tensor([0.0347, 0.0351, 0.0352, 0.0345], device='cuda:0',
       grad_fn=<SelectBackward>)
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
target  tensor([0], device='cuda:0')
Loss is  tensor(1.3865, device='cuda:0', grad_fn=<NllLossBackward>) 18 0
Last output  tensor([0.0347, 0.0351, 0.0352, 0.0345], device='cuda:0',
       grad_fn=<SelectBackward>)
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16])
shape in forward torch.Size([29, 1, 16]

KeyboardInterrupt: ignored